In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import glob
import skimage.io as io
import cv2
import os
from tqdm import tqdm

In [ ]:
paths = glob.glob("/root/data/gtsf_phase_I/*/*.json")

# DEBUG IMAGE LOAD

In [ ]:
paths = glob.glob('/root/data/gtsf_phase_I/2019*/*.json')

In [ ]:
class FLAGS(object):
    keypoints_order = ["TAIL_NOTCH",
                        "ADIPOSE_FIN",
                        "UPPER_LIP",
                        "ANAL_FIN",
                        "PELVIC_FIN",
                        "EYE",
                        "PECTORAL_FIN",
                        "DORSAL_FIN"]
    augmentation = None
    epochs=300
    crop = True
    input_size = (1024, 1024)

In [ ]:
def load_image_keypoints(annotation, FLAGS, reshape=True, buffer=100):
    """from annotation load image + keypoints"""
    # load image first
    if 'local_path' in annotation:
        local_path = annotation['local_path']
    else:
        local_path = os.path.join("/root/data/gtsf_phase_I/", 
                                  "/".join(annotation["Labeled Data"].split("/")[7:]))
    image = cv2.imread(local_path)
    #print(local_path)
       
    # load annotations second
    keypoints = []
    for kp_name in FLAGS.keypoints_order:
        value = annotation["Label"][kp_name]
        keypoints.append([int(value[0]["geometry"]["x"]), 
                          int(value[0]["geometry"]["y"])])
    if FLAGS.augmentation:
        transform = FLAGS.augmentation(image=image, 
                                       keypoints=keypoints)
        image = transform["image"]
        keypoints = transform["keypoints"]
    
    # crop the image min / max value
    keypoints = np.array(keypoints)
    height, width, _ = image.shape
    if FLAGS.crop:
        xs = keypoints[:, 0]
        min_x = np.max([np.min(xs) - buffer, 0])
        max_x = np.min([np.max(xs) + buffer, width])
        
        ys = keypoints[:, 1]
        min_y = np.max([np.min(ys) - buffer, 0])
        max_y = np.min([np.max(ys) + buffer, height])
        
#         print(min_y,max_y, min_x, max_x)
        image = image[min_y:max_y, min_x:max_x, : ]
#         print(image.shape)
    else:
        min_x = 0
        min_y = 0
        
    height, width, _ = image.shape
    if not reshape:
        ratio_width = 1.0
        ratio_height = 1.0
    else:
        ratio_width = width / FLAGS.input_size[0]
        ratio_height = height / FLAGS.input_size[1]
        image = cv2.resize(image, FLAGS.input_size)
    
    # let's recalculate the keypoints
    keypoints[:, 0] = (keypoints[:, 0] - min_x) / ratio_width
    keypoints[:, 1] = (keypoints[:, 1] - min_y) / ratio_height
      
    
    return image, keypoints

In [ ]:
paths = ['/root/data/gtsf_phase_I/2019-05-02/2019-05-02_cogito_annotations.json']
paths = glob.glob('/root/data/gtsf_phase_I/2019*/*.json')

In [ ]:
annotations = []
for jsonpath in tqdm(paths):
    annotations += json.load(open(jsonpath))

In [ ]:

for ann in tqdm(annotations):
    if not os.path.isfile(ann['local_path']):
        print('bug')
    local_path = ann['local_path']
    try:
        image, kps = load_image_keypoints(ann, FLAGS)
        shape = image.shape
    except Exception as e:
        print(e)
        # print(ann)
        print(local_path)


In [ ]:
image = cv2.imread('/root/data/gtsf_phase_I/2019-05-02/190502010013/rectified/right_small-pen-test-site_1_1556789575202.jpg')

# plot images

In [ ]:
!pip3 freeze | grep sci

In [ ]:
import random

from PIL import Image
from skimage.measure import label

In [ ]:
paths = ['/root/data/gtsf_phase_I/2019-05-02/2019-05-02_cogito_annotations.json']
paths = glob.glob('/root/data/gtsf_phase_I/2019*/*.json')

In [ ]:
annotations = []
for jsonpath in tqdm(paths):
    annotations += json.load(open(jsonpath))

In [ ]:
random.shuffle(annotations)

In [ ]:
def crop_and_mask(image_path, min_hsv, max_hsv):
    """remove green pixels"""
    name = os.path.basename(image_path)
    im = Image.open(image_path).resize((512, 512))
    im = np.array(im.convert('HSV'))
    h = np.array(im[...,0])
    s = np.array(im[...,1])
    v = np.array(im[...,2])
    
    mask = np.logical_and(h > min_hsv[0], h < max_hsv[0]) \
           & np.logical_and(s > min_hsv[1], s < max_hsv[1]) \
#            & np.logical_and(v > min_hsv[2], v < max_hsv[2]) 
    return mask, h,s,v

In [ ]:
# print(np.min(np.array(im)[100:150, 100:150, 0]))
# print(np.min(np.array(im)[100:150, 100:150, 1]))
# print(np.min(np.array(im)[100:150, 100:150, 2]))
# print('#')
# print(np.max(np.array(im)[100:150, 100:150, 0]))
# print(np.max(np.array(im)[100:150, 100:150, 1]))
# print(np.max(np.array(im)[100:150, 100:150, 2]))

In [ ]:
MIN_HSV = (50, 111, 56)
MAX_HSV = (120, 255, 89)

In [ ]:
image_path = '/root/data/gtsf_phase_I/2019-05-02/190502010036/rectified/right_small-pen-test-site_1_1556794876251.jpg'
image_path = '/root/data/gtsf_phase_I/2019-03-18/190318010048/rectified/right_small-pen-test-site_1_1552908763753.jpg'
image_path = '/root/data/gtsf_phase_I/2019-05-02/190502010015/rectified/right_small-pen-test-site_1_1556789855544.jpg'

In [ ]:
mask, h, _, _ = crop_and_mask(image_path, MIN_HSV, MAX_HSV)

In [ ]:
mask = np.abs(mask - 1)

In [ ]:
plt.imshow(mask)
plt.show()

mask = mask.astype(np.float64)
mask = cv2.erode(mask, np.ones((3,3))) # get ride of the noise
# mask = cv2.erode(mask, np.ones((3,3)))
mask = cv2.dilate(mask, np.ones((3,3)))
mask = cv2.dilate(mask, np.ones((3,3)))

plt.imshow(mask)
plt.show()

In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
def find_fish(mask):
    labels = label(mask)
    center = (256, 256)
    
    mindist = 1e5
    maxpoints = 0
    
    fishlabel = None
    for l in np.unique(labels):
        if l == 0:
            continue
        if np.count_nonzero(labels == l) < 500:
            continue

#         centroid = np.mean(np.where(labels == l), axis=1)
        distances = pairwise_distances(np.expand_dims(center, axis=0),
                                       np.array(np.where(labels == l)).transpose())
        close_point = np.count_nonzero(distances < 100)
#         dist = np.linalg.norm(center - centroid)
        if close_point > maxpoints:
            maxpoints = close_point
            fishlabel = l
#             plt.imshow(labels == l)
#             plt.title(l)
#             plt.show()
#     print(fishlabel)
    return np.array(labels == fishlabel)

In [ ]:
for (i, ann) in enumerate(annotations[20:40]):
    image_path = ann['local_path']
    print(i, image_path)
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    ratio_x = height / 512
    ratio_y = width / 512
    
    mask, _, _, _ = crop_and_mask(image_path, MIN_HSV, MAX_HSV)
    mask = np.abs(mask - 1)
    mask = mask.astype(np.float64)
    mask = cv2.erode(mask, np.ones((3,3)))
    mask = cv2.dilate(mask, np.ones((3,3)))
    mask = cv2.dilate(mask, np.ones((3,3)))

    fish_mask = find_fish(mask)
    
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    plt.imshow(mask)
    plt.show()
    
    plt.imshow(fish_mask)
    plt.show()
    
    # mask boundaries
    xs, ys = np.where(fish_mask > 0)
    x1, x2 = np.min(xs), np.max(xs)
    y1, y2 = np.min(ys), np.max(ys)
    
    # bring back to image scale
    x1 = int(x1*ratio_x)
    y1 = int(y1*ratio_y)
    x2 = int(x2*ratio_x)
    y2 = int(y2*ratio_y)
    
    # buffer
#     print(x1, x2, y1, y2)
    buffer = 300
    x1 = np.max([x1 - buffer, 0])
    x2 = np.min([x2 + buffer, height])
    y1 = np.max([y1 - buffer, 0])
    y2 = np.min([y2 + buffer, width])
#     print(x1, x2, y1, y2)
    
    plt.imshow(image[x1:x2, y1:y2, :])
    plt.show()
    
    print('#'*100)


In [ ]:
width, height = Image.open(image_path).size

In [ ]:
width

In [ ]:
height

# remove green

In [ ]:
img_path = '/root/data/phase_I/small-pen-test-site/1/2019-02-27/190227010006/raw/right_small-pen-test-site_1_1551258013966.jpg'
image = cv2.imread(img_path)
body = image[1200: 1900, 200:2500]
plt.figure(figsize=(20, 15))
plt.imshow(body)
plt.show()

# more stuff

In [ ]:
# img_path = np.random.choice(paths)
img_path = '/root/data/phase_I/small-pen-test-site/1/2019-02-27/190227010006/raw/right_small-pen-test-site_1_1551258013966.jpg'
image = cv2.imread(img_path, 0)
body = image[1300: 1800, 200:2500]
plt.figure(figsize=(20, 15))
plt.imshow(body)
plt.show()

In [ ]:
head = image[1450:1700, 300:500]
plt.imshow(head)
# plt.scatter(320, 10, color="r")
plt.show()

# keypoint matching

In [ ]:
import cv2

In [ ]:
orb = cv2.ORB_create()

In [ ]:
body_resized = body[:, :500]

In [ ]:
kp1, des1 = orb.detectAndCompute(head,None)
kp2, des2 = orb.detectAndCompute(body_resized,None)

In [ ]:
headkp = cv2.drawKeypoints(head, kp1, None)
plt.imshow(headkp)
plt.show()

In [ ]:
bodykp = cv2.drawKeypoints(body_resized, kp2, None)
plt.figure(figsize=(20, 10))
plt.imshow(bodykp)
plt.show()

In [ ]:
# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])
# cv.drawMatchesKnn expects list of lists as matches.
img3 = cv2.drawMatchesKnn(body_resized, kp2, head, kp1, good, None, flags=2)
plt.figure(figsize=(20, 10))
plt.imshow(img3),plt.show()
plt.show()